<a href="https://colab.research.google.com/github/silviolima07/hackathon_big_data/blob/main/HACKA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colunas do dataset de teste:

 - semana (número inteiro): número da semana (1 a 4 de janeiro/2023)
 - pdv (número inteiro): código do ponto de venda
 - produto (número inteiro): código do SKU
 - quantidade (número inteiro): previsão de vendas

In [1]:
!pip install parquet -q
!pip install pyarrow -q
!pip -q install duckdb -q

In [2]:
import pandas as pd
import numpy as np
import duckdb
import os
import pyarrow.parquet as pq
import pyarrow as pa # Import pyarrow core library

pd.set_option('display.max_columns', 60)
pd.options.mode.chained_assignment = None

import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['font.size'] = 24

from IPython.core.pylabtools import figsize

import seaborn as sns
sns.set(font_scale = 2)

from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer # Changed Imputer to SimpleImputer and imported from sklearn.impute
from sklearn.preprocessing import MinMaxScaler

from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_absolute_percentage_error

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer
import pandas as pd
import numpy as np

In [5]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
import numpy as np

# Modelo deve prever a quantidade com estas colunas no dataset df_teste, gerando a coluna quantidade

In [31]:
data = {'semana': [1,2,3, 4, 5],
        'pdv': [1023, 1045, 1023, 1088, 1010],
        'produto': [123,234, 456, 123, 550]}
df_teste = pd.DataFrame(data)

In [9]:
def missing_values_table(df):
        mis_val = df.isnull().sum()

        mis_val_percent = 100 * df.isnull().sum() / len(df)

        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})

        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)

        print ("Seu dataframe tem " + str(df.shape[1]) + " colunas.\n"
            "Há " + str(mis_val_table_ren_columns.shape[0]) +
              " colunas que possuem valores ausentes.")

        return mis_val_table_ren_columns

In [10]:
path='/content/drive/MyDrive/HACKA/'

# Ler os arquivos parquet com  pyarrow
- cada parte do arquivo parquet foi lida individualmente
- depois foram concatenadas
- a quantidade de linhas e colunas ficou igual a leitura com duckdb

In [11]:
# Get the directory of the current file
directory = os.path.dirname(path)

# List all parquet files in the directory that are part of the dataset
all_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.startswith('part-') and f.endswith('.parquet')]

#print("Arquivos Parquet encontrados:")
#for file in all_files:
    #print(file)
print(f'Lendo e concatenando registros...')

tables = []
for file in all_files:
    try:
        # Read each parquet file into a PyArrow table
        table = pq.read_table(file)
        tables.append(table)
        print("\nTables Pyarrow:\n", tables)
    except Exception as e:
        print(f"Error reading file {file}: {e}")

if tables:
    # Concatenate the PyArrow tables
    # Use the promote parameter to handle schema differences by promoting to a common type
    combined_table = pa.concat_tables(tables, promote=True)

    # Convert the combined PyArrow Table to a Pandas DataFrame
    df_pyarrow_combined = combined_table.to_pandas()

    print("\nDataset pandas lido e concatenado pelo PyArrow (todas as colunas):")
    print("Colunas:", df_pyarrow_combined.columns.tolist())
    print(df_pyarrow_combined.shape)
else:
    print("No tables were read.")

Lendo e concatenando registros...

Tables Pyarrow:
 [pyarrow.Table
pdv: string
premise: string
categoria_pdv: string
zipcode: int32
----
pdv: [["2204965430669363375","5211957289528622910","9024493554530757353","8659197371382902429","1400854873763881130",...,"5047149578012031122","8352177618955815208","7754182004766664314","9044929558186581877","8795399024619655297"]]
premise: [["On Premise","On Premise","Off Premise","On Premise","On Premise",...,"Off Premise","Off Premise","On Premise","On Premise","On Premise"]]
categoria_pdv: [["Mexican Rest","Hotel/Motel","Convenience","Restaurant","Restaurant",...,"Package/Liquor","Package/Liquor","Special Event","Other On Premise","Bar"]]
zipcode: [[30741,80011,80751,80439,30093,...,81501,80219,81004,81226,80205]]]

Tables Pyarrow:
 [pyarrow.Table
pdv: string
premise: string
categoria_pdv: string
zipcode: int32
----
pdv: [["2204965430669363375","5211957289528622910","9024493554530757353","8659197371382902429","1400854873763881130",...,"5047149578

In [13]:
df_pyarrow_combined.columns

Index(['pdv', 'premise', 'categoria_pdv', 'zipcode', 'produto', 'categoria',
       'descricao', 'tipos', 'label', 'subcategoria', 'marca', 'fabricante',
       'internal_store_id', 'internal_product_id', 'distributor_id',
       'transaction_date', 'reference_date', 'quantity', 'gross_value',
       'net_value', 'gross_profit', 'discount', 'taxes'],
      dtype='object')

In [12]:
df_pyarrow_combined.head()

,pdv,premise,categoria_pdv,zipcode,produto,categoria,descricao,tipos,label,subcategoria,marca,fabricante,internal_store_id,internal_product_id,distributor_id,transaction_date,reference_date,quantity,gross_value,net_value,gross_profit,discount,taxes
0,2204965430669363375,On Premise,Mexican Rest,30741.0,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
1,5211957289528622910,On Premise,Hotel/Motel,80011.0,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
2,9024493554530757353,Off Premise,Convenience,80751.0,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
3,8659197371382902429,On Premise,Restaurant,80439.0,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
4,1400854873763881130,On Premise,Restaurant,30093.0,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN


# Leitura dos arquivos parquet com duckdb
- esta leitura foi descontinuada pois o pyarrow ja fez isso.

In [12]:
"""
# (opcional) limite de memória e nº de threads
con = duckdb.connect()
con.execute("PRAGMA memory_limit='6GB';")   # ajuste conforme sua RAM
con.execute("PRAGMA threads=4;")             # ajuste conforme CPU

parquet_path = '/content/drive/MyDrive/HACKA/'
out_path     = "/content/sample_1M.parquet"                    # saída
number_of_lines = 8000000 # quantidade de linhas a serem lidas

# Get the directory of the current file
directory = os.path.dirname(parquet_path)

# List all parquet files in the directory that are part of the dataset
all_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.startswith('part-') and f.endswith('.parquet')]

print("Arquivos Parquet encontrados:")
for file in all_files:
    print(file)
print(f'Lendo {number_of_lines} de registros...')

# DICA: se você só precisa de algumas colunas, selecione-as para reduzir IO/RAM
cols = "*"  # ex.: "col1, col2, col3"

# Amostra uniforme exata de 7.000.000 lines, in streaming, com semente fixa (reprodutível)
# Using a glob pattern to read all parquet files in the directory and union by name
query = f"""
#COPY (
#  SELECT {cols}
#  FROM read_parquet('{parquet_path}*.parquet', union_by_name=True)
#  USING SAMPLE {number_of_lines} (reservoir, 42)
#) TO '{out_path}' (FORMAT PARQUET);
"""
con.execute(query)

# Read the sample into pandas/polars afterwards (now it's small):
import pandas as pd
df = pd.read_parquet(out_path)
print("\nDataset lido pelo DuckDB (todas as colunas):")
print("Colunas:", df.columns.tolist())
print(df.shape)
"""


'\ncon.execute(query)\n\n# Read the sample into pandas/polars afterwards (now it\'s small):\nimport pandas as pd\ndf = pd.read_parquet(out_path)\nprint("\nDataset lido pelo DuckDB (todas as colunas):")\nprint("Colunas:", df.columns.tolist())\nprint(df.shape)\n'

In [13]:
#df.columns.tolist()
"""
['pdv',
 'premise',
 'categoria_pdv',
 'zipcode',
 'internal_store_id',
 'internal_product_id',
 'distributor_id',
 'transaction_date',
 'reference_date',
 'quantity',
 'gross_value',
 'net_value',
 'gross_profit',
 'discount',
 'taxes',
 'produto',
 'categoria',
 'descricao',
 'tipos',
 'label',
 'subcategoria',
 'marca',
 'fabricante']
 """


"\n['pdv',\n 'premise',\n 'categoria_pdv',\n 'zipcode',\n 'internal_store_id',\n 'internal_product_id',\n 'distributor_id',\n 'transaction_date',\n 'reference_date',\n 'quantity',\n 'gross_value',\n 'net_value',\n 'gross_profit',\n 'discount',\n 'taxes',\n 'produto',\n 'categoria',\n 'descricao',\n 'tipos',\n 'label',\n 'subcategoria',\n 'marca',\n 'fabricante']\n "

In [14]:
#print("PyArrow DataFrame columns:")
#print("Shape?", df_pyarrow_combined.shape)
#print(df_pyarrow_combined.columns.tolist())

#print("\nDuckDB DataFrame columns:")
#print("Shape?", df.shape)
#print(df.columns.tolist())
"""
PyArrow DataFrame columns:
Shape? (6582209, 23)
['internal_store_id', 'internal_product_id', 'distributor_id', 'transaction_date', 'reference_date', 'quantity', 'gross_value', 'net_value', 'gross_profit', 'discount', 'taxes', 'produto', 'categoria', 'descricao', 'tipos', 'label', 'subcategoria', 'marca', 'fabricante', 'pdv', 'premise', 'categoria_pdv', 'zipcode']

DuckDB DataFrame columns:
Shape? (6582209, 23)
['pdv', 'premise', 'categoria_pdv', 'zipcode', 'internal_store_id', 'internal_product_id', 'distributor_id', 'transaction_date', 'reference_date', 'quantity', 'gross_value', 'net_value', 'gross_profit', 'discount', 'taxes', 'produto', 'categoria', 'descricao', 'tipos', 'label', 'subcategoria', 'marca', 'fabricante']
"""

"\nPyArrow DataFrame columns:\nShape? (6582209, 23)\n['internal_store_id', 'internal_product_id', 'distributor_id', 'transaction_date', 'reference_date', 'quantity', 'gross_value', 'net_value', 'gross_profit', 'discount', 'taxes', 'produto', 'categoria', 'descricao', 'tipos', 'label', 'subcategoria', 'marca', 'fabricante', 'pdv', 'premise', 'categoria_pdv', 'zipcode']\n\nDuckDB DataFrame columns:\nShape? (6582209, 23)\n['pdv', 'premise', 'categoria_pdv', 'zipcode', 'internal_store_id', 'internal_product_id', 'distributor_id', 'transaction_date', 'reference_date', 'quantity', 'gross_value', 'net_value', 'gross_profit', 'discount', 'taxes', 'produto', 'categoria', 'descricao', 'tipos', 'label', 'subcategoria', 'marca', 'fabricante']\n"

# Coluna e tipos esperados no dataset de teste e submissão
- coluna semana no ano (1 a 52)
- pdv (número inteiro): código do ponto de venda
- produto (número inteiro): código do SKU
- quantidade (número inteiro): previsão de vendas

In [14]:
df = df_pyarrow_combined.copy()

# Estatistica Descritiva

In [15]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
zipcode,14419.0,62676.248769,24052.659458,8107.000000,30341.000000,80202.000000,80535.000000,90920.000000
quantity,6560698.0,8.125893,80.493873,-1530.000000,1.000000,2.000000,4.000000,94230.000000
gross_value,6560698.0,122.751233,866.426496,-42672.898438,28.035000,42.099998,92.000000,604173.917786
net_value,6560698.0,119.123296,865.175682,-39848.000046,27.110323,40.770485,87.879002,604173.917786
gross_profit,6560698.0,21.930921,232.646984,-274396.000000,7.140242,10.510241,21.732000,274416.000000
discount,6560698.0,27.666835,384.762433,-13096.799805,0.000000,2.300000,8.700000,240082.965628
taxes,6560698.0,3.627938,11.309847,-4099.408691,0.180000,0.540000,3.039677,2073.237549


# Tipo de cada coluna

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6582209 entries, 0 to 6582208
Data columns (total 23 columns):
 #   Column               Dtype  
---  ------               -----  
 0   pdv                  object 
 1   premise              object 
 2   categoria_pdv        object 
 3   zipcode              float64
 4   produto              object 
 5   categoria            object 
 6   descricao            object 
 7   tipos                object 
 8   label                object 
 9   subcategoria         object 
 10  marca                object 
 11  fabricante           object 
 12  internal_store_id    object 
 13  internal_product_id  object 
 14  distributor_id       object 
 15  transaction_date     object 
 16  reference_date       object 
 17  quantity             float64
 18  gross_value          float64
 19  net_value            float64
 20  gross_profit         float64
 21  discount             float64
 22  taxes                float64
dtypes: float64(7), object(16)
memory

# Percentual de dados ausentes por coluna

In [17]:
missing_values_table(df)

Seu dataframe tem 23 colunas.
Há 23 colunas que possuem valores ausentes.


,Missing Values,% of Total Values
label,6576590,99.9
subcategoria,6575149,99.9
descricao,6575117,99.9
marca,6575117,99.9
tipos,6575117,99.9
categoria,6575117,99.9
produto,6575117,99.9
fabricante,6575117,99.9
pdv,6567790,99.8
premise,6567790,99.8


# Colunas pdv e produto apresentam 99% de dados ausentes.
# Serao substituidas por
- internal_store_id
- internal_product_id

In [18]:
colunas = ['internal_store_id',  'pdv',  'internal_product_id', 'produto', 'transaction_date','quantity']

In [19]:
missing_values_table(df[colunas])

Seu dataframe tem 6 colunas.
Há 6 colunas que possuem valores ausentes.


,Missing Values,% of Total Values
produto,6575117,99.9
pdv,6567790,99.8
internal_store_id,21511,0.3
internal_product_id,21511,0.3
transaction_date,21511,0.3
quantity,21511,0.3


# Ano  remover as linhas ausentes em quantity as linhas de outras colunas também foram removidas.

In [20]:
df = df.dropna(subset=["quantity"]).copy()
missing_values_table(df[colunas])

Seu dataframe tem 6 colunas.
Há 2 colunas que possuem valores ausentes.


,Missing Values,% of Total Values
pdv,6560698,100.0
produto,6560698,100.0


In [21]:
df.columns

Index(['pdv', 'premise', 'categoria_pdv', 'zipcode', 'produto', 'categoria',
       'descricao', 'tipos', 'label', 'subcategoria', 'marca', 'fabricante',
       'internal_store_id', 'internal_product_id', 'distributor_id',
       'transaction_date', 'reference_date', 'quantity', 'gross_value',
       'net_value', 'gross_profit', 'discount', 'taxes'],
      dtype='object')

In [22]:
df2 = df[['internal_store_id', 'internal_product_id', 'transaction_date', 'quantity']].copy()

# Coluna são renomeadas

In [23]:
df2.rename(columns={'internal_store_id': 'pdv', 'internal_product_id': 'produto', 'quantity': 'quantidade'}, inplace=True)

In [24]:
df2.isna().sum()

,0
pdv,0
produto,0
transaction_date,0
quantidade,0


# Extrair o numero da semana no ano a partir da data de transacao

In [25]:
df2['transaction_date'] = pd.to_datetime(df2['transaction_date'])
df2['semana'] = df2['transaction_date'].dt.isocalendar().week
del df2['transaction_date']

In [26]:
df2.columns

Index(['pdv', 'produto', 'quantidade', 'semana'], dtype='object')

In [27]:
df2

,pdv,produto,quantidade,semana
21511,7384367747233276219,328903483604537190,1.0,28
21512,3536908514005606262,5418855670645487653,6.0,12
21513,3138231730993449825,1087005562675741887,3.0,36
21514,3681167389484217654,1401422983880045188,129.0,36
21515,7762413312337359369,6614994347738381720,1.0,7
...,...,...,...,...
6582204,1439579822593789678,2523145330235258825,3.0,49
6582205,2357589266242432199,3148628429396585683,1.0,20
6582206,7245187752757348055,6668439675743950745,1.0,33
6582207,8952609383990245450,3894706280449257667,4.0,5


In [28]:
missing_values_table(df2)

Seu dataframe tem 4 colunas.
Há 0 colunas que possuem valores ausentes.


,Missing Values,% of Total Values


In [29]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6560698 entries, 21511 to 6582208
Data columns (total 4 columns):
 #   Column      Dtype  
---  ------      -----  
 0   pdv         object 
 1   produto     object 
 2   quantidade  float64
 3   semana      UInt32 
dtypes: UInt32(1), float64(1), object(2)
memory usage: 231.5+ MB


In [30]:
df2

,pdv,produto,quantidade,semana
21511,7384367747233276219,328903483604537190,1.0,28
21512,3536908514005606262,5418855670645487653,6.0,12
21513,3138231730993449825,1087005562675741887,3.0,36
21514,3681167389484217654,1401422983880045188,129.0,36
21515,7762413312337359369,6614994347738381720,1.0,7
...,...,...,...,...
6582204,1439579822593789678,2523145330235258825,3.0,49
6582205,2357589266242432199,3148628429396585683,1.0,20
6582206,7245187752757348055,6668439675743950745,1.0,33
6582207,8952609383990245450,3894706280449257667,4.0,5


In [32]:
df2.nunique()

,0
pdv,15086
produto,7092
quantidade,16449
semana,52


# Coluna quantidade tem valores negativos
# Outlier certamente.

In [33]:
df2.describe()

,quantidade,semana
count,6.560698e+06,6560698.0
mean,8.125893e+00,27.934699
std,8.049387e+01,14.337151
min,-1.530000e+03,1.0
25%,1.000000e+00,16.0
50%,2.000000e+00,29.0
75%,4.000000e+00,40.0
max,9.423000e+04,52.0


# Extrair apenas 10% dos dados

In [34]:
df_baseline = df2.sample(frac=0.1, random_state=42)
df_baseline.shape

(656070, 4)

# Baseline para referência

In [35]:
X = df_baseline.drop('quantidade', axis=1)
y = df_baseline['quantidade']
#
print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

(656070, 3)
(656070,)


In [36]:
X_train.isna().sum()

,0
pdv,0
produto,0
semana,0


In [37]:
X_test.isna().sum()

,0
pdv,0
produto,0
semana,0


In [38]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# ---------------------------
#  Pipeline

# Define categorical and numerical features
categorical_features = ['pdv', 'produto']
numerical_features = ['semana']

# Create a column transformer to apply different preprocessing steps to different columns
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', StandardScaler(), numerical_features)
    ],
    remainder='passthrough' # Keep other columns (if any) - not strictly necessary here
)


pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ("model", LinearRegression())
])

# ---------------------------
# 4. Avaliação com MAPE (cross-validation)
# ---------------------------
def wmape(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

# Definir scorer com MAPE (quanto menor, melhor)
#mape_scorer = make_scorer(wmape, greater_is_better=False)

#scores = cross_val_score(pipeline, X_train, y_train, cv=3, scoring=mape_scorer)

#print("WMAPE médio (cross-val):", -scores.mean())
#print("WMAPE por fold:", -scores)

"""
Resultado usando LinearRegression e 10% dos dados:
WMAPE médio (cross-val): 1.2183947001894604
WMAPE por fold: [1.20463883 1.21263043 1.23791484]
"""

# ---------------------------
# 5. Treinar modelo final e prever no teste
# ---------------------------
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print("WMAPE test:", wmape(y_test, y_pred))
#
# Novo dataset para submissao
# Em df_teste colunas pdv e  produto devem ser object.
df_teste['pdv'] = df_teste['pdv'].astype(str)
df_teste['produto'] = df_teste['produto'].astype(str)

yp_pred = pipeline.predict(df_teste)
df_teste['quantidade'] = yp_pred.astype(int)
df_teste.head()

WMAPE test: 1.182225700741042


,semana,pdv,produto,quantidade
0,1,1023,123,15
1,2,1045,234,15
2,3,1023,456,15
3,4,1088,123,15
4,5,1010,550,15


# Remover Outliers

In [39]:
df_com_outliers = df2.copy()
df_com_outliers.shape

(6560698, 4)

In [40]:
df_com_outliers.head(3)

,pdv,produto,quantidade,semana
21511,7384367747233276219,328903483604537190,1.0,28
21512,3536908514005606262,5418855670645487653,6.0,12
21513,3138231730993449825,1087005562675741887,3.0,36


In [41]:
df_com_outliers.describe()

,quantidade,semana
count,6.560698e+06,6560698.0
mean,8.125893e+00,27.934699
std,8.049387e+01,14.337151
min,-1.530000e+03,1.0
25%,1.000000e+00,16.0
50%,2.000000e+00,29.0
75%,4.000000e+00,40.0
max,9.423000e+04,52.0


# Modo padrão usando IQT,  Q1 e Q3

In [42]:
first_quartile = df_com_outliers['quantidade'].describe()['25%']
third_quartile = df_com_outliers['quantidade'].describe()['75%']

iqr = third_quartile - first_quartile

df_sem_outliers= df_com_outliers[(df_com_outliers['quantidade'] > (first_quartile - 1.5 * iqr)) &
            (df_com_outliers['quantidade'] < (third_quartile + 1.5 * iqr))].copy()

In [42]:
def remove_outliers(df):
    print("Antes:", df.shape)
    first_quartile = df_com_outliers['quantidade'].describe()['25%']
    third_quartile = df_com_outliers['quantidade'].describe()['75%']

    iqr = third_quartile - first_quartile

    df_sem_outliers= df_com_outliers[(df_com_outliers['quantidade'] > (first_quartile - 1.5 * iqr)) &
            (df_com_outliers['quantidade'] < (third_quartile + 1.5 * iqr))].copy()
    df_sem_outliers = df_sem_outliers[df_sem_outliers['quantidade'] >= 0]
    print("Depois:", df_sem_outliers.shape)
    return df_sem_outliers

In [43]:
df_sem_outliers = remove_outliers(df_com_outliers)

Antes: (6560698, 4)
Depois: (5668967, 4)


In [44]:
df_sem_outliers.describe()

,quantidade,semana
count,5.668967e+06,5668967.0
mean,2.122756e+00,27.480658
std,1.672486e+00,14.422918
min,0.000000e+00,1.0
25%,1.000000e+00,15.0
50%,1.000000e+00,28.0
75%,3.000000e+00,40.0
max,8.499990e+00,52.0


In [45]:
df_sem_outliers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5668967 entries, 21511 to 6582208
Data columns (total 4 columns):
 #   Column      Dtype  
---  ------      -----  
 0   pdv         object 
 1   produto     object 
 2   quantidade  float64
 3   semana      UInt32 
dtypes: UInt32(1), float64(1), object(2)
memory usage: 200.0+ MB


# Divisão de treino e teste a ser usado no treinamento e avaliação

In [46]:
X = df_sem_outliers.drop('quantidade', axis=1)
y = df_sem_outliers['quantidade']
#
print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

(5668967, 3)
(5668967,)


# Treinamento e avaliação WMAPE sem outliers

# LinearRegression

In [47]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# ---------------------------
#  Pipeline

# Define categorical and numerical features
categorical_features = ['pdv', 'produto']
numerical_features = ['semana']

# Pipelines para cada tipo
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

numerical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("scaler", StandardScaler())
])

# Create a column transformer to apply different preprocessing steps to different columns
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_transformer, numerical_features)
    ]
)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ("model", LinearRegression())
])

# ---------------------------
# 4. Avaliação com MAPE (cross-validation)
# ---------------------------

def wmape(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

# Definir scorer com WMAPE (quanto menor, melhor)
#mape_scorer = make_scorer(wmape, greater_is_better=False)

#scores = cross_val_score(pipeline, X_train, y_train, cv=3, scoring=mape_scorer)

#print("WMAPE médio (cross-val):", -scores.mean())
#print("WMAPE por fold:", -scores)

"""
Resultado usando LinearRegression e 10% dos dados:
WMAPE médio (cross-val): 0.4456727354526528
WMAPE por fold: [0.44531342 0.44669343 0.44501136]
"""

# ---------------------------
# 5. Treinar modelo final e prever no teste
# ---------------------------
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print("WMAPE test:", wmape(y_test, y_pred))
#
# Novo dataset para submissao
# Em df_teste colunas pdv e  produto devem ser object.
df_teste['pdv'] = df_teste['pdv'].astype(str)
df_teste['produto'] = df_teste['produto'].astype(str)
yp_pred = pipeline.predict(df_teste)
df_teste['quantidade'] = yp_pred.astype(int)
df_teste.head()

WMAPE test: 0.4301066465117743


,semana,pdv,produto,quantidade
0,1,1023,123,2
1,2,1045,234,2
2,3,1023,456,2
3,4,1088,123,2
4,5,1010,550,2


# RandomForestRegressor

In [ ]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import make_scorer

# -----------------------------
# Métrica WMAPE
# -----------------------------
def wmape(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

wmape_scorer = make_scorer(wmape, greater_is_better=False)  # menor = melhor

# -----------------------------
# Feature Engineer
# -----------------------------
class StatsFeatureEngineer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.stats_pdv_mean = None
        self.stats_prod_mean = None
        self.stats_semana_mean = None
        self.stats_pdv_median = None
        self.stats_prod_median = None
        self.stats_semana_median = None

    def fit(self, X, y=None):
        df = X.copy()
        df["quantidade"] = y

        # Médias
        self.stats_pdv_mean = df.groupby("pdv")["quantidade"].mean().rename("pdv_mean_qtd")
        self.stats_prod_mean = df.groupby("produto")["quantidade"].mean().rename("prod_mean_qtd")
        self.stats_semana_mean = df.groupby("semana")["quantidade"].mean().rename("semana_mean_qtd")

        # Medianas
        self.stats_pdv_median = df.groupby("pdv")["quantidade"].median().rename("pdv_median_qtd")
        self.stats_prod_median = df.groupby("produto")["quantidade"].median().rename("prod_median_qtd")
        self.stats_semana_median = df.groupby("semana")["quantidade"].median().rename("semana_median_qtd")

        return self

    def transform(self, X):
        df = X.copy()

        df = df.join(self.stats_pdv_mean, on="pdv")
        df = df.join(self.stats_prod_mean, on="produto")
        df = df.join(self.stats_semana_mean, on="semana")

        df = df.join(self.stats_pdv_median, on="pdv")
        df = df.join(self.stats_prod_median, on="produto")
        df = df.join(self.stats_semana_median, on="semana")

        return df.fillna(0)

# -----------------------------
# Pipeline
# -----------------------------
categorical_features = ["pdv", "produto"]
numerical_features = ["semana", "pdv_mean_qtd", "prod_mean_qtd", "semana_mean_qtd",
                      "pdv_median_qtd", "prod_median_qtd", "semana_median_qtd"]

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

numerical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("scaler", StandardScaler())
])

col_transformer = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_features),
        ("num", numerical_transformer, numerical_features)
    ]
)


pipeline = Pipeline(steps=[
    ("feature_engineer", StatsFeatureEngineer()),
    ("preprocessor", col_transformer),
    ("regressor", RandomForestRegressor(random_state=42))
])

# -----------------------------
# Cross-validation simples
# -----------------------------
scores = cross_val_score(pipeline, X_train, y_train, cv=3, scoring=wmape_scorer)
print("WMAPE médio (cross-val):", -scores.mean())
print("WMAPE por fold:", -scores)

# -----------------------------
# GridSearchCV
# -----------------------------
param_grid = {
    "regressor__n_estimators": [100, 200],
    "regressor__max_depth": [10, 20, None],
    "regressor__min_samples_split": [2, 5]
}

grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=3,
    scoring=wmape_scorer,
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)

print("Melhores hiperparâmetros:", grid_search.best_params_)
print("Melhor WMAPE (cv):", -grid_search.best_score_)

# -----------------------------
# Avaliação final no holdout
# -----------------------------
best_pipeline = grid_search.best_estimator_
y_pred = best_pipeline.predict(X_test)
print("WMAPE test:", wmape(y_test, y_pred))

# -----------------------------
# Submissão
# -----------------------------
# Em df_teste colunas pdv e  produto devem ser object.
df_teste['pdv'] = df_teste['pdv'].astype(str)
df_teste['produto'] = df_teste['produto'].astype(str)

y_submit = best_pipeline.predict(df_teste)
df_submit = df_teste[["semana", "pdv", "produto"]].copy()
df_submit["quantidade"] = np.round(y_submit).astype(int)
df_submit.to_csv("submission.csv", sep=";", index=False, encoding="utf-8")
print("Arquivo submission.csv gerado!")


# Pipeline final

In [ ]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import make_scorer

# -----------------------------
# Métrica WMAPE
# -----------------------------
def wmape(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

wmape_scorer = make_scorer(wmape, greater_is_better=False)

# -----------------------------
# Feature Engineer
# -----------------------------
class StatsFeatureEngineer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        df = X.copy()
        df["quantidade"] = y

        # Médias
        self.stats_pdv_mean = df.groupby("pdv")["quantidade"].mean().rename("pdv_mean_qtd")
        self.stats_prod_mean = df.groupby("produto")["quantidade"].mean().rename("prod_mean_qtd")
        self.stats_semana_mean = df.groupby("semana")["quantidade"].mean().rename("semana_mean_qtd")

        # Medianas
        self.stats_pdv_median = df.groupby("pdv")["quantidade"].median().rename("pdv_median_qtd")
        self.stats_prod_median = df.groupby("produto")["quantidade"].median().rename("prod_median_qtd")
        self.stats_semana_median = df.groupby("semana")["quantidade"].median().rename("semana_median_qtd")

        return self

    def transform(self, X):
        df = X.copy()
        df = df.join(self.stats_pdv_mean, on="pdv")
        df = df.join(self.stats_prod_mean, on="produto")
        df = df.join(self.stats_semana_mean, on="semana")
        df = df.join(self.stats_pdv_median, on="pdv")
        df = df.join(self.stats_prod_median, on="produto")
        df = df.join(self.stats_semana_median, on="semana")
        return df.fillna(0)

# -----------------------------
# Pipeline
# -----------------------------
# Colunas
categorical_features = ["pdv", "produto"]
numerical_features = ["semana", "pdv_mean_qtd", "prod_mean_qtd", "semana_mean_qtd",
                      "pdv_median_qtd", "prod_median_qtd", "semana_median_qtd"]

# Transformadores
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

numerical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("scaler", StandardScaler())
])

col_transformer = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_features),
        ("num", numerical_transformer, numerical_features)
    ]
)

# Pipeline completo
pipeline = Pipeline(steps=[
    ("feature_engineer", StatsFeatureEngineer()),
    ("preprocessor", col_transformer),
    ("regressor", RandomForestRegressor(random_state=42))
])

# -----------------------------
# Cross-validation
# -----------------------------
scores = cross_val_score(pipeline, X_train, y_train, cv=3, scoring=wmape_scorer)
print("WMAPE médio (cross-val):", -scores.mean())
print("WMAPE por fold:", -scores)

# -----------------------------
# GridSearchCV
# -----------------------------
param_grid = {
    "regressor__n_estimators": [100, 200],
    "regressor__max_depth": [10, 20, None],
    "regressor__min_samples_split": [2, 5]
}

grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=3,
    scoring=wmape_scorer,
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)
print("Melhores hiperparâmetros:", grid_search.best_params_)
print("Melhor WMAPE (cv):", -grid_search.best_score_)

# -----------------------------
# Avaliação final
# -----------------------------
best_pipeline = grid_search.best_estimator_
y_pred = best_pipeline.predict(X_test)
print("WMAPE test:", wmape(y_test, y_pred))

# -----------------------------
# Submissão
# -----------------------------
# Garante que pdv e produto do dataset de teste sejam strings para o pipeline
df_teste['pdv'] = df_teste['pdv'].astype(str)
df_teste['produto'] = df_teste['produto'].astype(str)

# Predição
y_submit = best_pipeline.predict(df_teste)

# Monta dataframe de submissão
df_submit = df_teste[["semana", "pdv", "produto"]].copy()
df_submit["quantidade"] = np.round(y_submit).astype(int)

# Converte todas colunas para inteiro antes de salvar
df_submit["semana"] = df_submit["semana"].astype(int)
df_submit["pdv"] = df_submit["pdv"].astype(int)
df_submit["produto"] = df_submit["produto"].astype(int)

# Salva CSV final para submissão
df_submit.to_csv("submission.csv", sep=";", index=False, encoding="utf-8")
print("Arquivo submission.csv gerado!")


In [ ]:
# ---------------------------
#  Pipeline

# Define categorical and numerical features
categorical_features = ['pdv', 'produto']
numerical_features = ['semana']

# Pipelines para cada tipo
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

numerical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("scaler", StandardScaler())
])

# Create a column transformer to apply different preprocessing steps to different columns
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_transformer, numerical_features)
    ]
)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ("model", RandomForestRegressor())
])

# ---------------------------
# 4. Avaliação com MAPE (cross-validation)
# ---------------------------

def wmape(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

# Definir scorer com WMAPE (quanto menor, melhor)
#mape_scorer = make_scorer(wmape, greater_is_better=False)

#scores = cross_val_score(pipeline, X_train, y_train, cv=3, scoring=mape_scorer)

#print("WMAPE médio (cross-val):", -scores.mean())
#print("WMAPE por fold:", -scores)

"""
Resultado usando LinearRegression e 10% dos dados:
WMAPE médio (cross-val): 0.4456727354526528
WMAPE por fold: [0.44531342 0.44669343 0.44501136]
"""

# ---------------------------
# 5. Treinar modelo final e prever no teste
# ---------------------------
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print("WMAPE test:", wmape(y_test, y_pred))
#
# Novo dataset para submissao
yp_pred = pipeline.predict(df_teste)
df_teste['quantidade'] = yp_pred.astype(int)
df_teste.head()

# Treinamento e avaliação com novas features criadas

In [ ]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def wmape(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

# -----------------------------
# Preprocessor customizado
# -----------------------------
class Preprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.stats_pdv = None
        self.stats_prod = None
        self.stats_semana = None

    def fit(self, X, y=None):
        # Calcula estatísticas no treino
        df = X.copy()
        df["quantidade"] = y

        # mean()
        self.stats_pdv_mean = df.groupby("pdv")["quantidade"].mean().reset_index().rename(columns={"quantidade": "pdv_mean_qtd"})
        self.stats_prod_mean = df.groupby("produto")["quantidade"].mean().reset_index().rename(columns={"quantidade": "prod_mean_qtd"})
        self.stats_semana_mean = df.groupby("semana")["quantidade"].mean().reset_index().rename(columns={"quantidade": "semana_mean_qtd"})

        # median()
        self.stats_pdv_median = df.groupby("pdv")["quantidade"].median().reset_index().rename(columns={"quantidade": "pdv_median_qtd"})
        self.stats_prod_median = df.groupby("produto")["quantidade"].median().reset_index().rename(columns={"quantidade": "prod_median_qtd"})
        self.stats_semana_median = df.groupby("semana")["quantidade"].median().reset_index().rename(columns={"quantidade": "semana_median_qtd"})

        return self

    def transform(self, X):
        df = X.copy()

        # Merge estatísticas
        df = df.merge(self.stats_pdv_mean, on="pdv", how="left")
        df = df.merge(self.stats_prod_mean, on="produto", how="left")
        df = df.merge(self.stats_semana_mean, on="semana", how="left")

        df = df.merge(self.stats_pdv_median, on="pdv", how="left")
        df = df.merge(self.stats_prod_median, on="produto", how="left")
        df = df.merge(self.stats_semana_median, on="semana", how="left")

        # Substitui NaN
        df = df.fillna(0)

        return df


# Cria pipeline de pré-processamento
preprocessor = Preprocessor()
X_train_transformed = preprocessor.fit_transform(X_train, y_train)
X_test_transformed = preprocessor.transform(X_test)

print("Colunas após o Preprocessor:")
print(X_train_transformed.columns.tolist())

# Define features automaticamente
categorical_features = ["pdv", "produto"]
numerical_features = [col for col in X_train_transformed.columns if col not in categorical_features]

# Pipelines para cada tipo
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

numerical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("scaler", StandardScaler())
])

# Create a column transformer to apply different preprocessing steps to different columns
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_transformer, numerical_features)
    ]
)

# ColumnTransformer
#column_transformer = ColumnTransformer(
#    transformers=[
#        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
#        ("num", StandardScaler(), numerical_features)
#    ]
#)

# Modelo final em pipeline
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", RandomForestRegressor(random_state=42))
])

# Treino
pipeline.fit(X_train_transformed, y_train)

# Avaliação
# Definir scorer com WMAPE (quanto menor, melhor)
#mape_scorer = make_scorer(wmape, greater_is_better=False)

#scores = cross_val_score(pipeline, X_train_transformed, y_train, cv=3, scoring=mape_scorer)

#print("WMAPE médio (cross-val):", -scores.mean())
#print("WMAPE por fold:", -scores)

# 5. Treinar modelo final e prever no teste
# ---------------------------
# Predição
y_pred = pipeline.predict(X_test_transformed)
y_pred = pipeline.predict(X_test)
print("WMAPE test:", wmape(y_test, y_pred))
#
# Novo dataset para submissao
yp_pred = pipeline.predict(df_teste)
df_teste['quantidade'] = yp_pred.astype(int)
df_teste.head()


# Treinamento e avaliação usando RandomForestRegressor
# Coluna com a media de venda por pdv criada.

In [ ]:
# ---------------------------
# 2. Transformer customizado
# ---------------------------
class MediaVendasPDV(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.media_por_pdv_ = pd.DataFrame({"pdv": X["pdv"], "Quantidade": y}).groupby("pdv")["Quantidade"].mean()
        self.media_global_ = self.media_por_pdv_.mean()
        return self

    def transform(self, X):
        X = X.copy()
        X["Media_Vendas_PDV"] = X["pdv"].map(self.media_por_pdv_)
        X["Media_Vendas_PDV"].fillna(self.media_global_, inplace=True)
        print("Dataset gerado:", X.columns.tolist())
        return X

# ---------------------------
# 3. Pipeline
# ---------------------------
categoricas = ['pdv', "produto"]
numericas = ["semana", "Media_Vendas_PDV"] # Added 'semana' and 'Media_Vendas_PDV' to numerical features

preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), categoricas),
    ("num", StandardScaler(), numericas)
])

pipeline = Pipeline([
    ("media_pdv", MediaVendasPDV()),
    ("preprocessor", preprocessor),
    ("model", RandomForestRegressor())
])

def wmape(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

# =======================================
# 4. Treinamento
# =======================================
modelo = RandomForestRegressor(
    n_estimators=200, random_state=42, n_jobs=-1
)
modelo.fit(X_train, y_train)

# =======================================
# 5. Avaliação
# =======================================
y_pred = modelo.predict(X_test)
print("WMAPE test:", wmape(y_test, y_pred))
"""
print("Processando cross_val_score...")
# Definir scorer com MAPE (quanto menor, melhor)
mape_scorer = make_scorer(wmape, greater_is_better=False)

scores = cross_val_score(pipeline, X_train, y_train, cv=3, scoring=mape_scorer)

print("WMAPE médio (cross-val):", -scores.mean())
print("WMAPE por fold:", -scores)
"""
"""
Resultado usando RandomForestRegressor e 10% dos dados:
WMAPE médio (cross-val): 0.4045400915671511
WMAPE por fold: [0.40469029 0.40565603 0.40327395]
"""
# Novo dataset para submissao
yp_pred = pipeline.predict(df_teste)
df_teste['quantidade'] = yp_pred.astype(int)
df_teste.head()

In [51]:
# =======================================
# 1. Função de WMAPE
# =======================================
def wmape(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

# =======================================
# 2. Custom Preprocessing Transformer
# =======================================
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline


class Preprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.pdv_mode = None
        self.produto_mode = None
        self.quantity_median = None
        self.train_stats = None  # To store stats for feature engineering
        self.train_semana_stats = None  # To store week stats from training

    def fit(self, X, y=None):
        # Ensure X is a DataFrame
        X = pd.DataFrame(X)

        # Store modes and median from the training data
        self.pdv_mode = X['pdv'].mode().iloc[0]
        self.produto_mode = X['produto'].mode().iloc[0]
        if 'quantidade' in X.columns:
            self.quantity_median = X['quantidade'].median()

        # Calculate and store train stats for feature engineering
        if 'quantidade' in X.columns and 'semana' in X.columns:
            self.train_stats = self._calculate_train_stats(X)
            # Specifically store week stats as 'semana' is in the test set
            if 'semana' in X.columns and pd.api.types.is_numeric_dtype(X['semana']):
                self.train_semana_stats = X.groupby("semana")["quantidade"].agg(
                    ["mean", "median", "std"]
                ).reset_index().rename(columns={
                    "mean": "semana_mean_qtd",
                    "median": "semana_median_qtd",
                    "std": "semana_std_qtd"
                })
            else:
                self.train_semana_stats = pd.DataFrame()  # Empty if cannot calculate

        return self

    def transform(self, X):
        # Ensure X is a DataFrame
        X = pd.DataFrame(X)

        # Apply the preprocessing steps
        X['pdv'] = X['pdv'].fillna(self.pdv_mode)
        X['produto'] = X['produto'].fillna(self.produto_mode)
        if 'quantidade' in X.columns:
            X['quantidade'] = X['quantidade'].fillna(self.quantity_median)

        # Handle 'semana' column
        if 'transaction_date' in X.columns:
            # Fill missing transaction_date before converting to datetime
            # Using mode from original training data if available, otherwise find mode in current X
            if self.train_stats and 'transaction_date_most_freq' in self.train_stats[2].columns:  # This check is incorrect, train_stats doesn't have transaction_date_most_freq
                # Need a way to get the mode of the original transaction_date if needed
                pass  # Skip this part for now, rely on filling NaT after conversion

            X['transaction_date'] = pd.to_datetime(X['transaction_date'], errors='coerce')
            # Handle potential NaT values after conversion
            if not X['transaction_date'].isnull().all():
                X['transaction_date'] = X['transaction_date'].fillna(X['transaction_date'].mode().iloc[0])
            else:
                # If all dates became NaT, fill with a dummy date
                X['transaction_date'] = X['transaction_date'].fillna(pd.to_datetime('2023-01-01'))

            X['semana'] = X['transaction_date'].dt.isocalendar().week.astype('int64')
            # Drop the original 'transaction_date'
            X = X.drop(columns=['transaction_date'])

        elif 'semana' in X.columns:
            # If 'semana' is already present, ensure it's int and use it directly
            X['semana'] = X['semana'].astype('int64')
        else:
            print("Warning: Neither 'transaction_date' nor 'semana' found. Cannot create/use 'semana' feature.")
            # Add 'semana' column with a default value or handle as needed
            X['semana'] = 0  # Add a default 'semana' to avoid errors later, adjust as needed


        # Apply feature engineering based on stored train_stats
        if self.train_stats:
            # Ensure semana stats are used if available
            self.train_stats = (self.train_stats[0], self.train_stats[1], self.train_semana_stats)  # Update with stored week stats
            X = self._apply_feature_engineering(X, self.train_stats)
        else:
            print("Warning: train_stats not available for feature engineering.")


        return X

    def _calculate_train_stats(self, df):
        # Agregações por pdv
        stats_pdv = df.groupby("pdv")["quantidade"].agg(
            ["mean", "median", "std"]
        ).reset_index().rename(columns={
            "mean": "pdv_mean_qtd",
            "median": "pdv_median_qtd",
            "std": "pdv_std_qtd"
        })

        # Agregações por produto
        stats_prod = df.groupby("produto")["quantidade"].agg(
            ["mean", "median", "std"]
        ).reset_index().rename(columns={
            "mean": "prod_mean_qtd",
            "median": "prod_median_qtd",
            "std": "prod_std_qtd"
        })

        # Return stats excluding week stats, which are handled separately
        return (stats_pdv, stats_prod, pd.DataFrame())  # Return empty DataFrame for week stats here


    def _apply_feature_engineering(self, df, train_stats):
        stats_pdv, stats_prod, stats_semana = train_stats

        # Merge features
        df = df.merge(stats_pdv, on="pdv", how="left")
        df = df.merge(stats_prod, on="produto", how="left")
        # Merge on 'semana' only if stats_semana is not empty
        if not stats_semana.empty and 'semana' in df.columns:  # Check if 'semana' is in df before merging
            df = df.merge(stats_semana, on="semana", how="left")
        elif 'semana' in df.columns:
            print("Warning: Skipping merge with week stats as it was not calculated or is empty.")


        # Fill NaNs introduced by merging (for new pdv/produto/semana in test)
        # Fill with 0 or a suitable placeholder
        df = df.fillna(0)  # Using 0 as a simple fill value

        return df


# =======================================
# 3. Pipeline
# =======================================
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer
import pandas as pd
import numpy as np


# Define features that will be present after Preprocessor and before OneHotEncoder/StandardScaler
# These should include the original features and the new features created by the Preprocessor
features_after_preprocessing = ['pdv', 'produto', 'semana', 'pdv_mean_qtd', 'pdv_median_qtd',
                                'pdv_std_qtd', 'prod_mean_qtd', 'prod_median_qtd', 'prod_std_qtd',
                                'semana_mean_qtd', 'semana_median_qtd', 'semana_std_qtd']


# Define categorical and numerical features for the ColumnTransformer
categorical_features_for_ct = ['pdv', 'produto']
numerical_features_for_ct = [col for col in features_after_preprocessing if col not in categorical_features_for_ct]


# Create a column transformer to apply different preprocessing steps to different columns
preprocessor_ct = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_for_ct),
        ('num', StandardScaler(), numerical_features_for_ct)
    ],
    remainder='passthrough' # Keep other columns (if any)
)

pipeline = Pipeline([
    ('custom_preprocessor', Preprocessor()), # Add the custom preprocessor
    ('column_transformer', preprocessor_ct), # Add the column transformer
    ("model", RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)) # Add the model
])


# =======================================
# Prepare data for the pipeline
# =======================================
# The pipeline expects the raw data with columns needed by the Preprocessor
# We will use the df_sem_outliers as the input to the pipeline for training
# Include 'transaction_date' and 'quantity' as they are needed by the Preprocessor
Xy_train_raw = df_sem_outliers[['pdv', 'produto', 'semana', 'quantidade']].copy() # Use 'semana' and 'quantity' from df_sem_outliers
y_train_raw = Xy_train_raw.pop('quantidade') # Separate the target variable


# Split the raw data for training the pipeline and evaluation
X_train_pipe, X_val_pipe, y_train_pipe, y_val_pipe = train_test_split(
    Xy_train_raw, y_train_raw, test_size=0.2, random_state=42
)


# =======================================
# 4. Treinamento
# =======================================
print("Treinando pipeline...")
pipeline.fit(X_train_pipe, y_train_pipe)


# =======================================
# 5. Avaliação
# =======================================
print("Avaliando pipeline com dados de validação...")
y_val_pred = pipeline.predict(X_val_pipe)
print("WMAPE validação:", wmape(y_val_pipe, y_val_pred))


# =======================================
# 6. Aplicando no dataset de teste
# =======================================
# Prepare the test dataset with the columns expected by the Preprocessor
# Based on df_teste structure: 'semana', 'pdv', 'produto'
df_teste_processed = df_teste[['semana', 'pdv', 'produto']].copy()

print("Previsões no dataset de teste...")
y_test_pred = pipeline.predict(df_teste_processed)

# Submissão estilo Kaggle
submissao = pd.DataFrame({
    "semana": df_teste_processed["semana"].astype(int), # Include semana and cast to int
    "pdv": df_teste_processed["pdv"].astype(int), # Cast pdv to int
    "produto": df_teste_processed["produto"].astype(int), # Cast produto to int
    "quantidade": y_test_pred.astype(int) # Rename and cast to int
})

# Ensure quantity is non-negative
submissao['quantidade'] = submissao['quantidade'].apply(lambda x: max(0, x))


submissao.to_csv("submissao.csv", index=False)
print("Arquivo submissao.csv gerado!")

Treinando pipeline...


ValueError: A given column is not a column of the dataframe